# Free Cash Flow

In [1]:
# Import key libraries, function and classes
import numpy as np
import pandas as pd

In [2]:
# Note
## (1) - Fiscal years ending December 31
## (2) - ($ in thousands)

In [3]:
# Columns 
columns = ['2017','2018','2019','2020','2021']

In [4]:
index = ['Tax rate', 'EBIT', '(-) Taxes', '(-) Capital expenditures', '(+) Depreciation', 'Free Cash Flow',
         '(+) Investment', 'Free Cash Flow pro forma for investment','Sales','(x) Sales multiple', 'Total equity value',
         '(-) Accured prefered dividends','(-) Preferred participation', 'Remaining equity value', 
         'Discount rate (CAPM)','(/) Probability of success',
         'Hurdle Rate','Discount factor', 'Discounted equity value', '(-) Investment need', 'Post-money equity value',
         '(-) Equity Investment', 'Pre-money equity value', 'Pre-money price/share']

In [5]:
# Initialise the free cash flow dataframe
df_free_cash_flow = pd.DataFrame(columns= columns,
                                 index=index)

In [6]:
# Fill the DataFrame with zero
df_free_cash_flow.fillna(0.0, inplace=True)

In [7]:
# Set the tax rate across all years to 35%
df_free_cash_flow.loc['Tax rate',:] = 0.35

In [8]:
# Set the EBIT based on financial analysis of the company
df_free_cash_flow.loc['EBIT',:]= [-500,-1000,1000,2500,4000]

In [9]:
# Calculate tax
df_free_cash_flow.loc['(-) Taxes',:] = df_free_cash_flow.loc['Tax rate',:]*df_free_cash_flow.loc['EBIT',:]

In [10]:
# If take is less than 0 set it to zero as you dont pay tax when EBIT is negative
df_free_cash_flow.loc['(-) Taxes',:] = [x if x >= 0 else 0.0 for x in df_free_cash_flow.loc['(-) Taxes',:].tolist()]

In [11]:
# Set the Capital expenditures
df_free_cash_flow.loc['(-) Capital expenditures',:] = [-500, -2000, -1500, -500, -500]

In [12]:
# Set the Depreciation
df_free_cash_flow.loc['(+) Depreciation',:] = [0, 500, 750, 750, 500]

In [13]:
# Calculate free cash flow
df_free_cash_flow.loc['Free Cash Flow',:] = df_free_cash_flow.loc['EBIT',:] + df_free_cash_flow.loc['(-) Taxes',:] + df_free_cash_flow.loc['(-) Capital expenditures',:] + df_free_cash_flow.loc['(+) Depreciation',:]

print(df_free_cash_flow)

                                            2017     2018     2019     2020  \
Tax rate                                    0.35     0.35     0.35     0.35   
EBIT                                     -500.00 -1000.00  1000.00  2500.00   
(-) Taxes                                   0.00     0.00   350.00   875.00   
(-) Capital expenditures                 -500.00 -2000.00 -1500.00  -500.00   
(+) Depreciation                            0.00   500.00   750.00   750.00   
Free Cash Flow                          -1000.00 -2500.00   600.00  3625.00   
(+) Investment                              0.00     0.00     0.00     0.00   
Free Cash Flow pro forma for investment     0.00     0.00     0.00     0.00   
Sales                                       0.00     0.00     0.00     0.00   
(x) Sales multiple                          0.00     0.00     0.00     0.00   
Total equity value                          0.00     0.00     0.00     0.00   
(-) Accured prefered dividends              0.00    

# Company Valuation 

In [14]:
# Note
## 1. Valuation started at the most far out year

In [15]:
# Enter desired sales - assume ~4x Revenue 
df_free_cash_flow.loc['Sales', '2021'] = 15000

In [16]:
# Assuming 2x sales multiple 
df_free_cash_flow.loc['(x) Sales multiple', '2021'] = 2

In [17]:
# Calculate the enterprise value 
df_free_cash_flow.loc['Total equity value', :] =  df_free_cash_flow.loc['(x) Sales multiple', :] * df_free_cash_flow.loc['Sales', :] 

In [18]:
# Assume most early companies have not net debt
# Hence, net debt = 0

## Pre / Post Money Valuation

In [19]:
# Discount rate (CAPM)
# Assumed for a stable big EM company 12% discount rate
df_free_cash_flow.loc['Discount rate (CAPM)', :] = .12

In [20]:
# Use the probability of success and failre to adjust the discount rate for a startup
# Assume the different failure and success rate per year
df_free_cash_flow.loc['(/) Probability of success', :] = [.25,.4,.6,.8,1.0]

In [21]:
# Hurdle rate = Discount rate (CAPM) / Probability of success
df_free_cash_flow.loc['Hurdle Rate', :] = df_free_cash_flow.loc['Discount rate (CAPM)', :]/df_free_cash_flow.loc['(/) Probability of success', :]

In [22]:
# Calculate the discount factor 
df_free_cash_flow.loc['Discount factor', '2017'] = 1/(1+df_free_cash_flow.loc['Hurdle Rate', '2017'])**4
df_free_cash_flow.loc['Discount factor', '2018'] = 1/(1+df_free_cash_flow.loc['Hurdle Rate', '2018'])**3
df_free_cash_flow.loc['Discount factor', '2019'] = 1/(1+df_free_cash_flow.loc['Hurdle Rate', '2019'])**2
df_free_cash_flow.loc['Discount factor', '2020'] = 1/(1+df_free_cash_flow.loc['Hurdle Rate', '2020'])**1
df_free_cash_flow.loc['Discount factor', '2021'] = 1/(1+df_free_cash_flow.loc['Hurdle Rate', '2021'])**0

In [23]:
# Calculate the discounted equity value 
## Total equity value in final year * discount rate
df_free_cash_flow.loc['Discounted equity value', :] = df_free_cash_flow.loc['Discount factor', :] * df_free_cash_flow.loc['Total equity value', '2021']

In [24]:
# Calculate the invstment needed  - how much of the fcf is negative... add total negative sum per year
## Manual input to save computational time 
df_free_cash_flow.loc['(-) Investment need', :] = [-3600,-2600,-100,0,0]

In [25]:
# Post-money equity valuation = Discount factor + (-) Investment need
df_free_cash_flow.loc['Post-money equity value', :] = df_free_cash_flow.loc['Discounted equity value', :] + df_free_cash_flow.loc['(-) Investment need', :]

## Capitalisation Table

In [26]:
# Note  - ($ in thousands, shares in millions)
# Initalise the capitalisation table 
cap_table = pd.DataFrame(index=['Management', 'Investor 1', 'Investor 2', 'Investor 3', 'Total' ], 
                         columns=['Series A - 2017', 'Series B - 2018', 'Series C - 2019', '2020', 'Exit - 2021' ])
cap_table.fillna(0.0, inplace=True)
cap_table

# In 2017, Business raised series A at £1m for investor 1
cap_table.loc['Investor 1', 'Series A - 2017'] = 1000
# In 2018, Business raised series B - investor 1 - £1m and investor 2 - £1.5m
cap_table.loc['Investor 1', 'Series B - 2018'] = 1000
cap_table.loc['Investor 2', 'Series B - 2018'] = 1500
# In 2019, Business raised series C from investor 3 £100k
cap_table.loc['Investor 3', 'Series C - 2019'] = 100

# Calculate the total
cap_table.loc['Total', :] = cap_table.loc['Investor 1', :] + cap_table.loc['Investor 2', :] + cap_table.loc['Investor 3', :] + cap_table.loc['Management', :]  
cap_table

,Series A - 2017,Series B - 2018,Series C - 2019,2020,Exit - 2021
Management,0.0,0.0,0.0,0.0,0.0
Investor 1,1000.0,1000.0,0.0,0.0,0.0
Investor 2,0.0,1500.0,0.0,0.0,0.0
Investor 3,0.0,0.0,100.0,0.0,0.0
Total,1000.0,2500.0,100.0,0.0,0.0


In [27]:
# Update the (- )equity investment with total capitalixation
df_free_cash_flow.loc['(-) Equity Investment', :] = -1*(cap_table.loc['Total', :].values)

In [28]:
# Calcualate Pre-money equity value
df_free_cash_flow.loc['Pre-money equity value', :] = df_free_cash_flow.loc['Post-money equity value', :] + df_free_cash_flow.loc['(-) Equity Investment', :]

### Capitalization Table - Share Issued

In [29]:
# note - shares in millions
shares_issued = pd.DataFrame(index=['Beginning shares outstanding', '(+) Shares Issued to Management', 
                                    '(+) Options issued to Management', '(+) Option pool', 
                                    '(+) Shares issued to Investor 1', '(+) Shares issued to Investor 2', 
                                    '(+) Shares issued to Investor 3', 'Ending shares outstanding'],
                             columns=['Series A - 2017', 'Series B - 2018', 'Series C - 2019', '2020', 'Exit - 2021'])
shares_issued.fillna(0.0, inplace=True)

# Build it 
## At the beginning of the company, no share issues
shares_issued.loc['Beginning shares outstanding', 'Series A - 2017'] = 0.0

# At the beginning, 1M shares was issued to management instead of pay 
shares_issued.loc['(+) Shares Issued to Management', 'Series A - 2017'] = 1

# At the beginning, 500K shares was issued as option to management
shares_issued.loc['(+) Options issued to Management', 'Series A - 2017'] = .5

# At the beginning, 500k shares was issued as option to employees
shares_issued.loc['(+) Option pool', 'Series A - 2017'] = .5


shares_issued

,Series A - 2017,Series B - 2018,Series C - 2019,2020,Exit - 2021
Beginning shares outstanding,0.0,0.0,0.0,0.0,0.0
(+) Shares Issued to Management,1.0,0.0,0.0,0.0,0.0
(+) Options issued to Management,0.5,0.0,0.0,0.0,0.0
(+) Option pool,0.5,0.0,0.0,0.0,0.0
(+) Shares issued to Investor 1,0.0,0.0,0.0,0.0,0.0
(+) Shares issued to Investor 2,0.0,0.0,0.0,0.0,0.0
(+) Shares issued to Investor 3,0.0,0.0,0.0,0.0,0.0
Ending shares outstanding,0.0,0.0,0.0,0.0,0.0


In [30]:
# The total shares before series a investment - Beginning shares outstanding + (+) Shares Issued to Management + +) Options issued to Management + (+) Option pool	
total_shares_b4_serisA = shares_issued.loc['Beginning shares outstanding', 'Series A - 2017'] + shares_issued.loc['(+) Shares Issued to Management', 'Series A - 2017'] + shares_issued.loc['(+) Options issued to Management', 'Series A - 2017'] + shares_issued.loc['(+) Option pool', 'Series A - 2017']


In [31]:
# Pre-money price per share = pre-money equity value at 2017 / total number of share before investment 
df_free_cash_flow.loc['Pre-money price/share','2017'] = df_free_cash_flow.loc['Pre-money equity value','2017']/ total_shares_b4_serisA/1000

In [32]:
# Investor 1 invested 1m in series  - amount invested by investor 1 / pre-money price per share
shares_issued.loc['(+) Shares issued to Investor 1', 'Series A - 2017']  =  cap_table.loc['Investor 1', 'Series A - 2017'] /  df_free_cash_flow.loc['Pre-money price/share','2017']/1000


In [33]:
# Investor 2 and 3 didnt make any investment in series A, 2017
# Ending shares outstanding 
shares_issued.loc['Ending shares outstanding', 'Series A - 2017'] = shares_issued.iloc[0:6, 0].values.sum()

In [34]:
# Beginning share outstanding = ending shares outstanding 
shares_issued.loc['Beginning shares outstanding', 'Series B - 2018'] = shares_issued.loc['Ending shares outstanding', 'Series A - 2017']

In [35]:
shares_issued

,Series A - 2017,Series B - 2018,Series C - 2019,2020,Exit - 2021
Beginning shares outstanding,0.000000,3.210069,0.0,0.0,0.0
(+) Shares Issued to Management,1.000000,0.000000,0.0,0.0,0.0
(+) Options issued to Management,0.500000,0.000000,0.0,0.0,0.0
(+) Option pool,0.500000,0.000000,0.0,0.0,0.0
(+) Shares issued to Investor 1,1.210069,0.000000,0.0,0.0,0.0
(+) Shares issued to Investor 2,0.000000,0.000000,0.0,0.0,0.0
(+) Shares issued to Investor 3,0.000000,0.000000,0.0,0.0,0.0
Ending shares outstanding,3.210069,0.000000,0.0,0.0,0.0


## Shared owned 

In [36]:
## Define the shares owned 
shares_owned = pd.DataFrame(index=['Management','Option pool','Investor 1', 'Investor 2', 'Investor 3'], 
                            columns = ['Series A - 2017', 'Series B - 2018', 'Series C - 2019', '2020', 'Exit - 2021'])
shares_owned.fillna(0.0, inplace=True)

In [37]:
## Shared owned by management in series A = shares issued to management + options issues to management
shares_owned.loc['Management', :] = shares_issued.loc['(+) Shares Issued to Management', :] + shares_issued.loc['(+) Options issued to Management', :] 
shares_owned.loc['Option pool', :] = shares_issued.loc['(+) Option pool', :]
shares_owned.loc['Investor 1', :] = shares_issued.loc['(+) Shares issued to Investor 1', :]
shares_owned.loc['Investor 2', :] = shares_issued.loc['(+) Shares issued to Investor 2', :]
shares_owned.loc['Investor 3', :] = shares_issued.loc['(+) Shares issued to Investor 3', :]
shares_owned.loc['Total shares outstanding', :] = shares_owned.loc['Management', :] +shares_owned.loc['Option pool', :] + shares_owned.loc['Investor 1', :] + shares_owned.loc['Investor 2', :] + shares_owned.loc['Investor 3', :]

In [ ]:
## 

In [38]:
shares_owned

,Series A - 2017,Series B - 2018,Series C - 2019,2020,Exit - 2021
Management,1.500000,0.0,0.0,0.0,0.0
Option pool,0.500000,0.0,0.0,0.0,0.0
Investor 1,1.210069,0.0,0.0,0.0,0.0
Investor 2,0.000000,0.0,0.0,0.0,0.0
Investor 3,0.000000,0.0,0.0,0.0,0.0
Total shares outstanding,3.210069,0.0,0.0,0.0,0.0


## Ownership

In [39]:
ownership = pd.DataFrame(index=['Management','Option pool','Investor 1', 'Investor 2', 'Investor 3'], 
                            columns = ['Series A - 2017', 'Series B - 2018', 'Series C - 2019', '2020', 'Exit - 2021'])
ownership.fillna(0.0, inplace=True)

In [40]:
# Ownership by management
ownership.loc['Management', :] = shares_owned.loc['Management', :]/shares_owned.loc['Total shares outstanding', :] 
# Ownership by stadd
ownership.loc['Option pool', :] = shares_owned.loc['Option pool', :]/shares_owned.loc['Total shares outstanding', :]
# Ownership by investor 1
ownership.loc['Investor 1', :] = shares_owned.loc['Investor 1', :]/shares_owned.loc['Total shares outstanding', :]
# Ownership by investor 2
ownership.loc['Investor 2', :] = shares_owned.loc['Investor 2', :]/shares_owned.loc['Total shares outstanding', :]
# Ownership by investor 3
ownership.loc['Investor 3', :] = shares_owned.loc['Investor 3', :]/shares_owned.loc['Total shares outstanding', :]

In [41]:
ownership.loc['Total Ownership', :] = ownership.sum(axis=0).values.tolist()

In [42]:
## From series B to Exit, management and investors agreed to set the ownership of the option pool to 15%
ownership.iloc[1, 1:] = .15

In [43]:
ownership*100

,Series A - 2017,Series B - 2018,Series C - 2019,2020,Exit - 2021
Management,46.727963,NaN,NaN,NaN,NaN
Option pool,15.575988,15.0,15.0,15.0,15.0
Investor 1,37.696049,NaN,NaN,NaN,NaN
Investor 2,0.000000,NaN,NaN,NaN,NaN
Investor 3,0.000000,NaN,NaN,NaN,NaN
Total Ownership,100.000000,0.0,0.0,0.0,0.0


In [ ]:
## Switch decision
# Excel is a better tool for this sort of modelling at my cu

In [44]:
df_free_cash_flow

,2017,2018,2019,2020,2021
Tax rate,0.350000,0.350000,0.350000,0.350000,0.35
EBIT,-500.000000,-1000.000000,1000.000000,2500.000000,4000.00
(-) Taxes,0.000000,0.000000,350.000000,875.000000,1400.00
(-) Capital expenditures,-500.000000,-2000.000000,-1500.000000,-500.000000,-500.00
(+) Depreciation,0.000000,500.000000,750.000000,750.000000,500.00
Free Cash Flow,-1000.000000,-2500.000000,600.000000,3625.000000,5400.00
(+) Investment,0.000000,0.000000,0.000000,0.000000,0.00
Free Cash Flow pro forma for investment,0.000000,0.000000,0.000000,0.000000,0.00
Sales,0.000000,0.000000,0.000000,0.000000,15000.00
(x) Sales multiple,0.000000,0.000000,0.000000,0.000000,2.00
